# GMF

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [24]:
class GMF(nn.Module):
    """
    
    Note: Convert input user_id and item_id to 0-based indices
    """
    def __init__(self, n_users, n_items, embed_dim):
        super().__init__()
        self.user_embed = nn.Embedding(n_users, embed_dim)
        self.item_embed = nn.Embedding(n_items, embed_dim)
        self.linear = nn.Linear(embed_dim, 1)
        self.sigmoid = nn.Sigmoid()
        
        self._init_weight()
        
    def _init_weight(self):
        nn.init.normal_(self.user_embed.weight, std=0.01)
        nn.init.normal_(self.item_embed.weight, std=0.01)
        nn.init.normal_(self.linear.weight, std=0.01)
    
    def forward(self, users, items):
        users -= 1
        items -= 1
        user_embeds = self.user_embed(users)
        item_embeds = self.item_embed(items)
        out = user_embeds * item_embeds
        out = self.linear(out)
        out = self.sigmoid(out) * 5
        return out

In [26]:
from typing import Union
def train(model: nn.Module, 
          dataloader: DataLoader, 
          criterion: nn.Module, 
          optimizer: torch.optim.Optimizer, 
          device: Union[torch.device, str]):
    model.train()
    for users, items, ratings in dataloader: 
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)
        # Zero out the grad
        optimizer.zero_grad()
        
        # Predict the ratings
        outputs = model(users, items)
        
        # Calculate loss
        loss = criterion(outputs, ratings)
        
        # Calculate gradient descent
        loss.backward()
        
        # Update weight
        optimizer.step()

def evaluation(model: nn.Module, 
               dataloader: DataLoader, 
               criterion: nn.Module, 
               device: Union[torch.device, str]):     
        model.eval()
        total_loss = 0
        error = 0
        total_ratings = 0
        with torch.no_grad():
            for users, items, ratings in dataloader:
                users, items = users.to(device), items.to(device)
                outputs = model(users, items)
                loss = criterion(outputs, ratings)
                total_loss += loss
                
                error += ((outputs-ratings)**2).sum().item()
                total_ratings += ratings.size(0)
        
        return total_loss / len(dataloader), torch.sqrt(error/total_ratings)        

In [27]:
from utils import RatingDataset
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.001
for i in range(1, 6):
    training_filepath = f'../../data/ml-100k/u{i}.base'
    testing_filepath = f'../../data/ml-100k/u{i}.test'
    
    training_dataset = RatingDataset(training_filepath)
    testing_dataset = RatingDataset(testing_filepath)
    
    training_dataloader = DataLoader(training_dataset, batch_size=32, shuffle=True)
    testing_dataloader = DataLoader(testing_dataset, batch_size=32, shuffle=False)
    
    model = GMF(943, 1682, 64)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train(model, training_dataloader, criterion, optimizer, device)
    loss, rmse = evaluation(model, testing_dataloader, criterion, device)
    print(f"Dataset {i}: Loss: {loss}, RMSE: {rmse}")
    

c:\Users\T14s G4\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not float